In [41]:
# autoreload
%load_ext autoreload
%autoreload 2

# fix path
import sys
sys.path.append("/home/jxm/reverse-training")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
from utils.analysis import load_results_as_df
import os


HOME_DIR = "/home/jxm/reverse-training"
df = load_results_as_df(os.path.join(HOME_DIR, "results"))
# df = df[df["select_seed_dataset"].str.endswith("200000")]
# df = df[df["select_seed_dataxset"].str.endswith("500000")]
# df = df[df["select_label_strategy"] == "auto"]
# df = df[df["select_num_pseudoexperts"] == 1]
print("Length:", len(df))


Filtered 0/144 results with no evaluation metrics
dict_keys(['data', 'full_ot_distance', 'sinkhorn_distance', 'jaccard_overlap_examples', 'jaccard_overlap_vocabulary', 'levenshtein_stats', 'args', 'expert_evaluation_metrics', 'evaluation_metrics'])
Length: 144


In [43]:
df["select_batch_fill_strategy"].value_counts()

select_batch_fill_strategy
greedy_batched    32
topk              30
random            30
bottomk           30
topk_balanced     22
Name: count, dtype: int64

In [44]:
print(df["dataset"].value_counts(), "\n")
print(df["select_full_dataset_size"].value_counts(), "\n")
print(df["select_batch_fill_strategy"].value_counts(), "\n")
print(df["select_num_pseudoexperts"].value_counts(), "\n")

dataset
ag_news_10000      50
dbpedia_10000      47
newsgroup_10000    47
Name: count, dtype: int64 

select_full_dataset_size
100      31
1000     30
500      30
5000     27
10000    25
1500      1
Name: count, dtype: int64 

select_batch_fill_strategy
greedy_batched    32
topk              30
random            30
bottomk           30
topk_balanced     22
Name: count, dtype: int64 

select_num_pseudoexperts
1    144
Name: count, dtype: int64 



In [45]:
df["select_projection_dim"].value_counts()

select_projection_dim
4096    143
1024      1
Name: count, dtype: int64

In [46]:
import pandas as pd

pd.set_option('display.max_rows', None)

dv_cols = ["dataset", "select_seed_dataset", "select_full_dataset_size", "select_batch_fill_strategy"]

iv_cols = ["best_eval_accuracy", "dataset_sinkhorn_distance", "dataset_jaccard_overlap_vocabulary"]

final_df = df
# final_df = df[df["select_seed_dataset"] == "nq_500000"]

final_df.groupby(dv_cols)[iv_cols].min().sort_values(by=["dataset","select_seed_dataset", "select_full_dataset_size", "select_batch_fill_strategy"], ascending=False)

best_eval_accuracy  \
dataset         select_seed_dataset select_full_dataset_size select_batch_fill_strategy                       
newsgroup_10000 nq_20000            10000                    topk                                  0.297852   
                                                             random                                0.287109   
                                                             greedy_batched                        0.284180   
                                                             bottomk                               0.295898   
                                    5000                     topk                                  0.285156   
                                                             random                                0.252930   
                                                             greedy_batched                        0.254883   
                                                             bottomk                               0.279297   
                                    1000                     topk_balanced                         0.181641   
                                                             topk                                  0.207031   
                                                             random                                0.170898   
                                                             greedy_batched                        0.206055   
                                                             bottomk                               0.209961   
                                    500                      topk_balanced                         0.112305   
                                                             topk                                  0.138672   
                                                             random                                0.125000   
                                                             greedy_batched                        0.148438   
                                                             bottomk                               0.143555   
                                    100                      topk_balanced                         0.063477   
                                                             topk                                  0.058594   
                                                             random                                0.051758   
                                                             greedy_batched                        0.054688   
                                                             bottomk                               0.058594   
                nq_100000           1500                     greedy_batched                        0.236328   
dbpedia_10000   nq_20000            10000                    topk                                  0.833008   
                                                             random                                0.833008   
                                                             greedy_batched                        0.805664   
                                                             bottomk                               0.816406   
                                    5000                     topk_balanced                         0.828125   
                                                             topk                                  0.779297   
                                                             random                                0.791992   
                                                             greedy_batched                        0.772461   
                                                             bottomk                               0.781250   
                                    1000                     topk_balanced                         0.703125   
                                                             topk                            